<img src="logo.png">

# siuba.dply.vector

``siuba.dply.vector`` se trata de una familia de funciones auxiliares que complementan a los verbos de siuba que hemos estudiado. Se trata de funciones que se aplican a series, y por lo tanto a columnas de dataframes. En https://siuba.org/api/dply.vector puedes ver la documentación completa.

In [ ]:
import numpy as np
import pandas as pd
from siuba import *
from siuba.dply.vector import *

Tomemos el siguiente dataframe ventas de nuestro repositorio:

In [ ]:
ventas = pd.read_csv("C:\\Users\\Usuario\\Documents\\scidata\\24_cdd_py\\practicas\\ventas\\ventas.csv",
                    encoding="utf8")
ventas

In [ ]:
pd.set_option('display.max_rows', None)
ventas

In [ ]:
#`between`: Seleccionar filas donde ventas están entre 200 y 230

ventas >> filter(_.ventas.between(200,230))

In [ ]:
#`cumall`: Indicar si todas las ventas hasta la fila actual son mayores a 150
ventas >> mutate(salida = cumall(_.ventas>150))

In [ ]:
#`cumany`: Indicar si alguna venta hasta la fila actual es mayor a 205
ventas >> mutate(salida = cumany(_.ventas>230))

In [ ]:
## Calcular las ventas acumuladas 
ventas >> mutate(ventas_acumuladas =  np.nancumsum(_.ventas))

In [ ]:
## Calcular las ventas acumuladas por cada región mensualmente
(
ventas >> group_by(_.region,_.mes) >> 
    summarize(total = _.ventas.sum()) >>
    group_by(_.region) >>
    mutate(suma_acum = np.nancumsum(_.total)) 
)    

In [ ]:
# `dense_rank`: Calcular el rango denso de ventas
ventas >> mutate(resultado = dense_rank(_.ventas)) 

In [ ]:
# `first`: Obtener el primer valor de ventas de cada región

ventas >> group_by(_.region) >> summarize(resultado = first(_.ventas))

In [ ]:
# `last`: Obtener el último valor de ventas

ventas >> group_by(_.region) >> summarize(resultado = last(_.ventas))

In [ ]:
# `lag`: Obtener la venta del período anterior
ventas >> mutate(lag_venta = _.ventas - lag(_.ventas))

In [ ]:
(ventas >> 
    group_by(_.region,_.mes) >> 
    summarize(ventas_tot = _.ventas.sum()) >> 
    mutate(futuro = lag(_.ventas_tot),
           dif_futuro=_.ventas_tot-_.futuro)
)

In [ ]:
# `lead`: Obtener la venta del período siguiente

ventas >> mutate(lead_venta = _.ventas - lead(_.ventas))

In [ ]:
(ventas >> 
    group_by(_.region,_.mes) >> 
    summarize(ventas_tot = _.ventas.sum()) >> 
    mutate(futuro = lead(_.ventas_tot),
           dif=_.ventas_tot-_.futuro)
)

In [ ]:
# `min_rank`: Calcular el rango mínimo de ventas
ventas >> mutate(min_rank_venta = min_rank(_.ventas))

In [ ]:
# `n`: Obtener el número total de filas
ventas >> group_by(_.region) >> summarize(total_filas = n(_))

In [ ]:
# `n_distinct`: Obtener el número de valores distintos en `producto`
#ventas >> summarize(num_distinct_productos = n_distinct(_.producto))
ventas >> n_distinct(_.producto)



In [ ]:
# `na_if`: Reemplazar el valor 200 en `ventas` con NaN
ventas >> mutate(ventas_na_if = na_if(_.ventas, 205))

In [ ]:
# `percent_rank`: Calcular el percentil de ventas
ventas  >> mutate(percent_rank_ventas = 100*percent_rank(_.ventas))

In [ ]:
np.percentile(ventas >> filter(-_.ventas.isna()) >> select(_.ventas),96.923077)

In [ ]:
# `row_number`: Asignar el número de fila a cada venta
ventas >> group_by(_.region) >> mutate(row_number_venta = row_number(_))